In [1]:
#DO MIXTRAL

import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

!pip install maritalk -q
!pip install langchain[all] -q

clear_output()

import sys,os,glob,json,sqlite3,re
import pandas as pd
from datetime import datetime
import pickle

sys.path.append("mixtral-offloading")

import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer, TFBertForSequenceClassification, TFXLMRobertaForSequenceClassification,TFDebertaV2ForSequenceClassification
from transformers.utils import logging as hf_logging

from src.build_model import OffloadConfig, QuantConfig, build_model

from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.language_models.chat_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage,
    BaseMessage
)
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

import maritalk

import numpy as np

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


In [2]:
class SingletonMeta(type):
    """
    The Singleton class can be implemented in different ways in Python. Some
    possible methods include: base class, decorator, metaclass. We will use the
    metaclass because it is best suited for this purpose.
    """

    _instances = {}

    def __call__(cls, *args, **kwargs):
        """
        Possible changes to the value of the `__init__` argument do not affect
        the returned instance.
        """
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

#CLASSES UTILITARIAS

In [4]:
class UtilsData():
  def __init__(self):
    with open('/content/drive/MyDrive/PECE/models/dataset_train_test/dataset_dict.pickle', 'rb') as file:
        dataset_dict = pickle.load(file)

    self.num_classes = dataset_dict['n_class']
    self.dict_classes = {str(k):v for k,v in enumerate(dataset_dict['labels'])}

    self.describe_tables = self.load_json(r"/content/drive/MyDrive/PECE/resoucers/dicionario_dados.json")
    self.intents_table = self.load_json(r"/content/drive/MyDrive/PECE/resoucers/AdventureQI.json")
    super().__init__()

  def load_json(self,file):
      f = open(file)
      dict = json.load(f)
      f.close()
      return dict

In [5]:
class UtilsDB(metaclass = SingletonMeta):
  def __init__(self):
    self.path_scripts = r"/content/drive/MyDrive/PECE/scripts"
    self.path_data = r"/content/drive/MyDrive/PECE/dados"
    self.conn = sqlite3.connect(r"/content/drive/MyDrive/PECE/db/dbo.db")

    super().__init__()
    pass

  def consulta_db(self,sql):
    try:
      df = pd.read_sql(sql,self.conn)
      resposta = df.to_dict("records")
    except:
      resposta = f"ERRO SQL: {sql}"
    return resposta



In [6]:
class UtilsChat(UtilsData,UtilsDB):
  def __init__(self):
    super().__init__()
    pass

  def captura_consulta(self,lista):
      regex = r"((?:(((`{1,3}|\\n)([Ss][Qq][Ll]))|(```))(\s|\\n|```))|(?=[Ss][Ee][Ll][Ee][Cc][Tt]))(.*?)(;|`{1,3}|$)"
      sql_query = lista[0].split("###RESPOSTA")[-1]

      for i in re.findall(regex,sql_query, re.DOTALL):
        return i[7]

  def cria_sql(self,question,table,fields):
    return self.chain_llm.invoke({"question":question,"table":",".join(table),"fields":fields})

  def cria_resposta(self,question,results):
    return self.chain_chat.invoke({"question":question,"result":results})

  def interacao_chat(self,question):
    intention = self.intent_recognition.identify_intention(question)
    intention = self.intent_recognition.identify_intention(question)
    intention = self.intent_recognition.identify_intention(question)

    table_for_intention = self.intents_table[intention]
    columns_for_table = str({table:self.describe_tables[table]['Colunas'] for table in table_for_intention})
    sql = self.cria_sql(question,table_for_intention,columns_for_table)

    print("Resposta Gerativo",sql)
    if isinstance(sql,list):
      sql= self.captura_consulta(sql)

    if sql == None:
        resultado = "Nenhum"
        resposta = self.cria_resposta(question,"Não foi possivel localizar")
        print("QUESTAO   >>>>> {}\nINTENÇÃO  >>>>> {} \nTABELA    >>>>> {} \nQUERY     >>>>> {} \nRESULTADO >>>>> {}".format(question,intention,table_for_intention,sql,str(resultado)))
        return resposta

    resultado = self.consulta_db(sql.replace("\\",""))
    resposta = self.cria_resposta(question,str(resultado))
    if isinstance(resposta,list):
      resposta = resposta[0].split("[/INST]")[-1]

    print("QUESTAO   >>>>> {}\nINTENÇÃO  >>>>> {} \nTABELA    >>>>> {} \nQUERY     >>>>> {} \nRESULTADO >>>>> {}".format(question,intention,table_for_intention,sql,str(resultado)))
    return resposta.split(".")[0]



  def interacao_chat_teste(self,question):
    intention1 = self.intent_recognition.identify_intention(question)

    dict_saida = dict()
    intention = intention1
    table_for_intention = self.intents_table[intention]
    columns_for_table = str({table:self.describe_tables[table]['Colunas'] for table in table_for_intention})
    sql = self.cria_sql(question,table_for_intention,columns_for_table)
    resposta_gerada = sql
    print("Questao",question)
    if isinstance(sql,list):
      sql= self.captura_consulta(sql)


    if sql == None:
        resultado = "Nenhum"
        resposta = self.cria_resposta(question,"Não foi possivel localizar")
        # print("QUESTAO   >>>>> {}\nINTENÇÃO  >>>>> {} \nTABELA    >>>>> {} \nQUERY     >>>>> {} \nRESULTADO >>>>> {}".format(question,intention,table_for_intention,sql,str(resultado)))
    else:
      resultado = self.consulta_db(sql.replace("\\",""))
      resposta = self.cria_resposta(question,str(resultado))

    if isinstance(resposta,list):
        resposta = resposta[0].split("[/INST]")[-1]

    # print("QUESTAO   >>>>> {}\nINTENÇÃO  >>>>> {} \nTABELA    >>>>> {} \nQUERY     >>>>> {} \nRESULTADO >>>>> {}".format(question,intention,table_for_intention,sql,str(resultado)))


    dict_saida ={"Questao":question,
                         "Intencao":intention,
                         "Gerativo":resposta_gerada,
                         "Query Gerada":sql,
                         "Resultado gerado":resultado,
                         "Resposta":resposta.split(".")[0]}



    return dict_saida



  def avaliacao_gerativos(self,query,question,intention):
    resultado_query_gerativo = list()
    resultado_query_teste = list()
    total_questoes = len(question)
    for i, quest in enumerate(question):
      print("{} de {}".format(i,total_questoes))

      table_for_intention = self.intents_table[intention[i]]
      columns_for_table = str({table:self.describe_tables[table]['Colunas'] for table in table_for_intention})
      sql = self.cria_sql(quest,table_for_intention,columns_for_table)
      resposta_gerada = sql
      if isinstance(sql,list):
        sql= self.captura_consulta(sql)

      if sql == None:
        resultado = "Nenhum"
      else:
        resultado = self.consulta_db(sql.replace("\\","").replace("-","_"))

      resultado_query_gerativo.append({ "Questao":quest,
                                       "Intencao":intention[i],
                                        "Query teste":query[i],
                                        "Gerativo":resposta_gerada,
                                        "Query Gerada":sql,
                                        "Resultado teste":self.consulta_db(query[i].replace("\\","")),
                                        "Resultado gerado":resultado})
      # resultado_query_teste.append(self.consulta_db(query[i].replace("\\","")))


    return resultado_query_gerativo


  def interacao_teste(self,question):
    intention = self.intent_recognition.identify_intention(question)
    table_for_intention = self.intents_table[intention]
    columns_for_table = str({table:self.describe_tables[table]['Colunas'] for table in table_for_intention})
    sql = self.cria_sql(question,table_for_intention,columns_for_table)

    if isinstance(sql,list):
      sql= self.captura_consulta(sql)

    if sql == None:
        resultado = "Nenhum"
        return intention, resultado

    resultado = self.consulta_db(sql.replace("\\",""))

    return intention, resultado


#INTENT RECOGINITION

In [7]:
class UtilsBert(UtilsData):
  def __init__(self):
    self.thresold_multilabel = 0.50
    super().__init__()
    pass

  def tokenization(self,sent):
    input_ids,attention_masks=list(),list()
    bert_inp=self.tokenizer.encode_plus(sent, add_special_tokens = True, max_length =64, pad_to_max_length = True, return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])
    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    return input_ids, attention_masks

  def identify_intention(self,sent,response="label"):
    input,attention = self.tokenization(sent)
    preds = self.model.predict([input,attention],batch_size=1, verbose=0)

    if response == "label":
      return self.label_classification(preds)
    else:
      return self.multilabel_classification(preds)

  def multilabel_classification(self,preds):
    labels = list()
    idx=np.where(preds.logits>=self.thresold_multilabel,1,0)
    for i,col in enumerate(idx[0]):
      if col == 1:
        labels.append((self.dict_classes[str(i)],preds.logits[0][i]))
    return labels

  def label_classification(self,preds):
    pred_labels = np.argmax(preds.logits, axis=1)
    return self.dict_classes[str(pred_labels[0])]

In [8]:
class FineTuningBertimbau(UtilsBert,metaclass=SingletonMeta):
  def __init__(self):
    super().__init__()

    self.model_id = 'neuralmind/bert-base-portuguese-cased'
    self.weights = '/content/drive/MyDrive/PECE/models/bertimbau_model_v4.h5'

    self.tokenizer =  AutoTokenizer.from_pretrained(self.model_id)
    self.model = TFBertForSequenceClassification.from_pretrained(self.model_id,num_labels=self.num_classes)
    self.model.load_weights(self.weights)

In [9]:
class FineTuningRoBERTaXLM(UtilsBert, metaclass=SingletonMeta):
  def __init__(self):
    super().__init__()

    self.model_id = "xlm-roberta-large"
    self.weights = '/content/drive/MyDrive/PECE/models/xlmroberta_model_v5.h5'

    self.tokenizer =  AutoTokenizer.from_pretrained(self.model_id)
    self.model = TFXLMRobertaForSequenceClassification.from_pretrained(self.model_id, num_labels=self.num_classes,num_hidden_layers =12)
    self.model.load_weights(self.weights)

In [10]:
class FineTuningaAlBERTina(UtilsBert, metaclass=SingletonMeta):
  def __init__(self):
    super().__init__()

    self.model_id = 'PORTULAN/albertina-900m-portuguese-ptbr-encoder-brwac'
    self.weights = '/content/drive/MyDrive/PECE/models/albertina_model_v4.h5'

    self.tokenizer =  AutoTokenizer.from_pretrained(self.model_id)
    self.model = TFDebertaV2ForSequenceClassification.from_pretrained(self.model_id,num_labels=self.num_classes,num_hidden_layers =12)
    self.model.load_weights(self.weights)

#AGENTS TEXT-TO-SQL

In [11]:
class ChatMaritalk(SimpleChatModel):

    api_key: str
    temperature: float = 0.7
    chat_mode: bool = True
    max_tokens: int = 512
    do_sample: bool = True
    top_p: float = 0.95
    system_message_workaround: bool = True

    @property
    def _llm_type(self) -> str:
        return "maritalk"

    def parse_messages_for_model(self, messages: List[BaseMessage]):
        parsed_messages=[]

        for message in messages:
            if isinstance(message, HumanMessage):
                parsed_messages.append({"role":"user","content":message.content})
            elif isinstance(message, AIMessage):
                parsed_messages.append({"role":"assistant","content":message.content})
            elif isinstance(message, SystemMessage):
                if self.system_message_workaround:
                    parsed_messages.append({"role":"user","content":message.content})
                    parsed_messages.append({"role":"assistant","content": "ok"})

        return parsed_messages

    def _call(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        print(self.temperature)
        model = maritalk.MariTalk(key=self.api_key)
        stop_tokens=stop if stop is not None else []
        messages=self.parse_messages_for_model(messages)
        answer = model.generate(
            messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            stopping_tokens=stop_tokens,
            do_sample=self.do_sample,
            top_p=self.top_p,
            chat_mode=True,
        )

        return answer

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"system_message_workaround": self.system_message_workaround, "temperature": self.temperature, "top_p": self.top_p, "max_tokens": self.max_tokens }

In [12]:
class AgentMaritaca(UtilsChat):

  def __init__(self,model_intent ,**kwargs):
    super().__init__()
    self.intent_recognition = model_intent()

    self.temperature_llm = kwargs.get('temperature_llm',0.7)
    self.temperature_chat = kwargs.get('temperature_llm',0.7)

    self.API_KEY = "115462282570231901569$ba212bf19c4a16c8d1fd183e17e64279534d9eaf7fe0c70288c7c0e04324e46c"
    self.llm = ChatMaritalk(api_key=self.API_KEY,chat_mode=False,temperature=self.temperature_llm,max_tokens=100)
    self.chat = ChatMaritalk(api_key=self.API_KEY,chat_mode=True,temperature=self.temperature_chat,max_tokens=30)
    self.output_parser = StrOutputParser()


    self.llm_prompt_template = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente especialista em construir instruções SQL. Crie uma query SQL que atenda a solicitação levando em conta as tabelas envolvidas e os campos"""),
        ("human", """Solicitação: {question}, Tabelas: {table}, Campos:{fields}""")
    ])
    self.chat_prompt_template = ChatPromptTemplate.from_messages([
        ("system", """Você é uma assistente especialista atender solicitações de informações com informações extraidas de banco de dados. . Responda a solicitação com o resultado fornecido."""),
        ("human", """Solicitação:{question}, Resultado:{result}"""),
    ])
    self.chain_llm = self.llm_prompt_template | self.llm | self.output_parser
    self.chain_chat = self.chat_prompt_template | self.chat | self.output_parser


In [13]:
class AgentMaritacaMOD(UtilsChat):

  def __init__(self,model_intent1 ,model_intent2 ,model_intent3 ,**kwargs):
    super().__init__()
    self.intent_recognition1 = model_intent1()
    self.intent_recognition2 = model_intent2()
    self.intent_recognition3 = model_intent3()

    self.temperature_llm = kwargs.get('temperature_llm',0.7)
    self.temperature_chat = kwargs.get('temperature_llm',0.7)

    self.API_KEY = "115462282570231901569$ba212bf19c4a16c8d1fd183e17e64279534d9eaf7fe0c70288c7c0e04324e46c"
    self.llm = ChatMaritalk(api_key=self.API_KEY,chat_mode=False,temperature=self.temperature_llm,max_tokens=100)
    self.chat = ChatMaritalk(api_key=self.API_KEY,chat_mode=True,temperature=self.temperature_chat,max_tokens=30)
    self.output_parser = StrOutputParser()


    self.llm_prompt_template = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente especialista em construir instruções SQL. Crie uma query SQL que atenda a solicitação levando em conta as tabelas envolvidas e os campos"""),
        ("human", """Solicitação: {question}, Tabelas: {table}, Campos:{fields}""")
    ])
    self.chat_prompt_template = ChatPromptTemplate.from_messages([
        ("system", """Você é uma assistente especialista atender solicitações de informações com informações extraidas de banco de dados. Responda a solicitação EM PORTUGUÊS com o resultado fornecido."""),
        ("human", """Solicitação:{question}, Resultado:{result}"""),
    ])
    self.chain_llm = self.llm_prompt_template | self.llm | self.output_parser
    self.chain_chat = self.chat_prompt_template | self.chat | self.output_parser

In [14]:
class BaseModelMixtral(metaclass = SingletonMeta):

    def __init__(self, **kwargs: Any):
        model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
        quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
        state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

        config = AutoConfig.from_pretrained(quantized_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side='left')
        device = torch.device("cuda:0")
        offload_per_layer = 1

        num_experts = config.num_local_experts

        offload_config = OffloadConfig(
            main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
            offload_size=config.num_hidden_layers * offload_per_layer,
            buffer_size=4,
            offload_per_layer=offload_per_layer,
        )


        attn_config = BaseQuantizeConfig(
            nbits=4,
            group_size=64,
            quant_zero=True,
            quant_scale=True,
        )
        attn_config["scale_quant_params"]["group_size"] = 256


        ffn_config = BaseQuantizeConfig(
            nbits=2,
            group_size=16,
            quant_zero=True,
            quant_scale=True,
        )
        quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


        self.model = build_model(
            device=device,
            quant_config=quant_config,
            offload_config=offload_config,
            state_path=state_path,
        )



In [15]:
class ChatMixtral(SimpleChatModel):

    temperature: float = 0.7
    max_tokens: int = 512
    do_sample: bool = True
    top_p: float = 0.95
    system_message_workaround: bool = True
    device = torch.device("cuda:0")

    @property
    def _llm_type(self) -> str:
      return "mixtral"

    def __init__(self,**kwargs):
      super().__init__()
      self.temperature = kwargs.get("temperature",self.temperature)
      self.max_tokens = kwargs.get("max_tokens",self.max_tokens)
      self.do_sample = kwargs.get("do_sample",self.do_sample)
      self.top_p = kwargs.get("top_p",self.top_p)
      self.system_message_workaround = kwargs.get("system_message_workaround",self.system_message_workaround)

    def parse_messages_for_model(self, messages: List[BaseMessage]):
        parsed_messages=[]

        for message in messages:
            # print(message,isinstance(message, AIMessage))
            if isinstance(message, HumanMessage):
                parsed_messages.append({"role":"user","content":message.content})
            elif isinstance(message, AIMessage):
                parsed_messages.append({"role":"assistant","content":message.content})
            elif isinstance(message, SystemMessage):
                if self.system_message_workaround:
                    parsed_messages.append({"role":"user","content":message.content})
                    parsed_messages.append({"role":"assistant","content": "ok"})

        return parsed_messages

    def _call(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        print
        messages=self.parse_messages_for_model(messages)
        mixtral = BaseModelMixtral()
        input_ids = mixtral.tokenizer.apply_chat_template(messages, return_tensors="pt").to(self.device)
        attention_mask = torch.ones_like(input_ids)
        answer = mixtral.model.generate(
              input_ids=input_ids,
              attention_mask=attention_mask,
              do_sample=self.do_sample,
              temperature=self.temperature,
              top_p=self.top_p,
              max_new_tokens=self.max_tokens,
              pad_token_id=mixtral.tokenizer.eos_token_id
              )


        decoded = mixtral.tokenizer.batch_decode(answer, skip_special_tokens=True)
        return decoded


    @property
    def _identifying_params(self) -> Mapping[str, Any]:
      """Get the identifying parameters."""
      return {"system_message_workaround": self.system_message_workaround, "temperature": self.temperature, "top_p": self.top_p, "max_tokens": self.max_tokens }



In [16]:
class AgentMixtral(UtilsChat):

  def __call__(self,**kwargs):
    self.temperature_llm = kwargs.get('temperature_llm',0.4)
    self.temperature_chat = kwargs.get('temperature_llm',0.7)

  def __init__(self,model_intent ,**kwargs):
    super().__init__()
    self.intent_recognition = model_intent()
    self.temperature_llm = kwargs.get('temperature_llm',0.4)
    self.temperature_chat = kwargs.get('temperature_llm',0.7)

    print("TEMPERATURA",self.temperature_llm)

    self.llm = ChatMixtral(temperature=self.temperature_llm,max_tokens=500)
    self.chat = ChatMixtral(temperature=self.temperature_chat,max_tokens=50,)
    self.output_parser = StrOutputParser()


    self.llm_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente especialista em construir instruções SQL. Crie uma query SQL no SQLITE que atenda a solicitação levando em conta SOMENTE as tabelas e campos enviados"""),
        ("human", """###Solicitação: {question} ###Tabelas: {table} ###Campos:{fields}"""),
        ("ai", """###RESPOSTA: """)])

    self.chat_prompt =ChatPromptTemplate.from_messages([
        ("system", """"Você é um assistente especialista em atender solicitações com dados extraidos de bancos de dados. Responda a solicitação EM PORTUGUÊS com o resultado fornecido."""),
        ("human", """###SOLICITAÇÃO:{question} ###RESULTADO:{result}""")])


    self.chain_llm = self.llm_prompt | self.llm | self.output_parser
    self.chain_chat = self.chat_prompt | self.chat | self.output_parser

In [17]:
class AgentMixtralMOD(UtilsChat):

  def __call__(self,**kwargs):
    self.temperature_llm = kwargs.get('temperature_llm',0.4)
    self.temperature_chat = kwargs.get('temperature_llm',0.7)

  def __init__(self,model_intent1 ,model_intent2,model_intent3,**kwargs):
    super().__init__()
    self.intent_recognition1 = model_intent1()
    self.intent_recognition2 = model_intent2()
    self.intent_recognition3 = model_intent3()
    self.temperature_llm = kwargs.get('temperature_llm',0.4)
    self.temperature_chat = kwargs.get('temperature_llm',0.7)

    print("TEMPERATURA",self.temperature_llm)

    self.llm = ChatMixtral(temperature=self.temperature_llm,max_tokens=500)
    self.chat = ChatMixtral(temperature=self.temperature_chat,max_tokens=50,)
    self.output_parser = StrOutputParser()


    self.llm_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente especialista em construir instruções SQL. Crie uma query SQL que atenda a solicitação levando em conta tabelas e campos enviados"""),
        ("human", """###Solicitação: {question} ###Tabelas: {table} ###Campos:{fields}"""),
        ("ai", """###RESPOSTA: """)])

    self.chat_prompt =ChatPromptTemplate.from_messages([
        ("system", """"Você é um assistente especialista em atender solicitações com dados extraidos de bancos de dados. Responda a solicitação EM PORTUGUÊS com o resultado fornecido."""),
        ("human", """###SOLICITAÇÃO:{question} ###RESULTADO:{result}""")])


    self.chain_llm = self.llm_prompt | self.llm | self.output_parser
    self.chain_chat = self.chat_prompt | self.chat | self.output_parser

In [18]:
import pickle
with open('/content/drive/MyDrive/PECE/models/dataset_train_test/dataset_dict.pickle', 'rb') as file:
    dataset_dict = pickle.load(file)

num_classes = dataset_dict['n_class']
labels = dataset_dict['labels']

df = pd.read_excel("/content/drive/MyDrive/PECE/resoucers/querys.xlsx",sheet_name='Planilha1')
df_to_use = df.dropna()
data=df_to_use.rename(columns = {'NEW INTENT': 'label', 'Question': 'text'}, inplace = False)

df_questoes = pd.DataFrame(dataset_dict['X_test'].values,columns=['Questao'])
querys = df_questoes.merge(data,left_on=['Questao'],right_on=['text'],how='left')['SQL'].to_list()
questoes = dataset_dict['X_test'].values
intencoes = [labels[label_index] for label_index in dataset_dict['y_test'].tolist()]

In [19]:
# agent = AgentMixtral(FineTuningBertimbau,temperature_llm=0.4)
# mixtral_2 = agent.avaliacao_gerativos(querys,questoes,intencoes)

# with open('/content/drive/MyDrive/PECE/models/mixtral_2', 'wb') as fp:
#   pickle.dump(mixtral_2, fp)



#USO

In [20]:
# question = "Ocupacao do lucas piazzon"

In [21]:
# bertimbau = FineTuningBertimbau()

# print(bertimbau.identify_intention(question))
# print(bertimbau.identify_intention(question,response="multilabel"))

In [22]:
# mixtralAgent = AgentMixtral(FineTuningBertimbau,temperature_llm=0.4)
# maritacaAgent = AgentMaritaca(FineTuningBertimbau,temperature_llm=0.3)

In [23]:
questions = ["Cargo de Susan Eaton",
             "Em que cargo Suchitra Mohan esta?",
             "Christopher Hill é de que departamento?",
             "De quanto é a renda anual do cliente Arthur Sanchez",
             "Quantos departamentos existem na empresa?",
            "Qual o cargo do Paulo Jorge?",
            "Em qual departamento o Joao Castilho está?",
            "De quanto é a renda anual do cliente Alex Fernando",
            "Quantidade de carros do Juliana Ahuig",
            "Renda Anual cliente Carina Ramos"
             ]

In [24]:
mixtralAgentMOD = AgentMixtral(FineTuningBertimbau,
                                  temperature_llm=0.7)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TEMPERATURA 0.7


In [30]:
maritacaAgent =  AgentMaritaca(FineTuningBertimbau,
                                  temperature_llm=0.7)

In [31]:
saidas = list()
for quest in questions:
  saidas.append(maritacaAgent.interacao_chat_teste(quest))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.7
Questao Cargo de Susan Eaton
0.7


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.7
Questao Em que cargo Suchitra Mohan esta?
0.7


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

0.7
Questao De quanto é a renda anual do cliente Alex Fernando
0.7


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.7
Questao Quantidade de carros do Juliana Ahuig
0.7


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.7
Questao Renda Anual cliente Carina Ramos
0.7


In [32]:
saidas

[{'Questao': 'Cargo de Susan Eaton',
  'Intencao': 'CARGO_DO_FUNCIONARIO',
  'Gerativo': " SELECT Title\nFROM DimEmployee\nWHERE EmployeeKey = 'SUSAN\\_EATON\\_EMPLOYEE\\_KEY'",
  'Query Gerada': " SELECT Title\nFROM DimEmployee\nWHERE EmployeeKey = 'SUSAN\\_EATON\\_EMPLOYEE\\_KEY'",
  'Resultado gerado': [],
  'Resposta': ' Desculpe, não tenho informações sobre o cargo de Susan Eaton'},
 {'Questao': 'Em que cargo Suchitra Mohan esta?',
  'Intencao': 'CARGO_DO_FUNCIONARIO',
  'Gerativo': " SELECT Title FROM DimEmployee WHERE FirstName = 'Suchitra' AND LastName = 'Mohan';",
  'Query Gerada': " SELECT Title FROM DimEmployee WHERE FirstName = 'Suchitra' AND LastName = 'Mohan';",
  'Resultado gerado': [{'Title': 'Production Technician - WC60'}],
  'Resposta': 'According to the information I have, Suchitra Mohan is currently a Production Technician - WC60'},
 {'Questao': 'Christopher Hill é de que departamento?',
  'Intencao': 'DEPARTAMENTO_DO_FUNCIONARIO',
  'Gerativo': " SELECT Department

In [27]:
regex = r"((?:(((```|\\n)([Ss][Qq][Ll]))|(```))(\s|\\n|```))|(?=[Ss][Ee][Ll][Ee][Cc][Tt]))(.*?)(;|```|$)"
sql_query = saidas[5]["Gerativo"][0].split("###RESPOSTA")[-1]

for i in re.findall(regex,sql_query, re.DOTALL):
  print(i[7])

SELECT Title 
FROM DimEmployee 
WHERE FirstName = 'Paulo' AND LastName = 'Jorge'

SELECT` pode retornar zero, um ou múltiplos registros, dependendo do número de ocorrências da cláusula `WHERE`. No caso de "Paulo Jorge" ser um funcionário único, a instrução retornará apenas uma linha com o seu respectivo cargo.


In [29]:
with open('/content/drive/MyDrive/PECE/models/geracao_resposta_maritaca', 'wb') as fp:
  pickle.dump(saidas, fp)


In [28]:
# from google.colab import runtime
# runtime.unassign()